In [1]:
import os
import numpy as np
import pandas as pd
from time import process_time
from itertools import product
from multiprocess import Pool
import threading
from dust_disk_integration import integration

In [2]:
# input variables

Degree_To_Rad = np.pi/180.
yr = np.pi*1e7 # 365*24*3600 # yr in [s]

k1_array = [1.] # m_star/m_Sun
k2_array = [1.] # m_planet/m_Jupiter
k_ap_array = np.array([10]) # a_Planet/R_Sun
beta_array = [0.15] # F_rad/F_grav
k_init = np.array([3.]) # a_initial/a_Planet
e_p = [0.] # planet eccentricity
e_initial = np.array([0.]) # initial dust particle eccentricity (=0)
inc_initial = np.array([0.*Degree_To_Rad]) # initial dust particle eccentricity (=0)
Omega_initial = [0.] # longitude of ascending node
pomega_initial = [0.] # initial dust longitude of periapsis

endtime = np.array([300 * yr]) # integration time
N_dust = np.array([100])


input_array = []
r = product(k1_array, k2_array, k_ap_array, beta_array, k_init, e_p, e_initial, inc_initial, Omega_initial, pomega_initial, endtime, N_dust)
for ri in r:
    input_array.append(ri)


In [3]:
from tabulate import tabulate

text = """
All subprocesses done.
"""

output = tabulate([[text]], tablefmt='grid')

In [4]:
# create an empty dataframe
paralabels = ['m_Star/m_Sun', 'm_Planet/m_J', 'a_p/R_Sun', 'R_sub/R_Sun', 'R_Planet/R_Sun',
              'beta', 'a_d_i/a_p', 'hash', 'a_initial', 'e_initial', 'inc_initial',
              'Omega_initial', 'pomega_initial', 'M_initial', 'CJ_array', 'a_d_array',
              'e_d_array', 'CJ_final', 'x_p_f_xyz', 'v_p_f_xyz', 'x_d_f_xyz', 'v_d_f_xyz',
              'final_fate', 'lifetime', 'a_d_f', 'e_d_f', 'vrel_f[km/s]', 'mu']
df_para = pd.DataFrame(columns=paralabels)

In [5]:
%%time

if __name__ == "__main__":
                
    pool = Pool(processes=128) # pool number = cpu number by default

    for pool_output in pool.imap(integration, input_array):
        # merge data
        df_para = pd.concat([df_para, pool_output], axis=0)
        df_para = df_para.reset_index(drop = True)
        # print ('index of data:', len(df_para))

    # save data
    df_para = df_para.sort_values(by=['m_Star/m_Sun', 'm_Planet/m_J', 'a_p/R_Sun'], ascending = False)
    df_para.to_csv('./test.csv', header=True, index=False)
    
    pool.close() # close() doesn't kill any process; it just closes a pipe which informs that there will be no more data coming through it.
    pool.join() # Killed processes send a signal informing their parents that they are quite dead.
    
    print (output)


Sublimation: dust 93 at 103.09637 [yr]
Ejection: dust 100 at 104.02010 [yr]
Sublimation: dust 87 at 104.35180 [yr]
Hit the Planet: dust 99 at 104.35676 [yr]
Sublimation: dust 9 at 104.62817 [yr]
Hit the Planet: dust 96 at 105.19894 [yr]
Hit the Planet: dust 3 at 105.27296 [yr]
Hit the Planet: dust 101 at 105.56579 [yr]
Hit the Planet: dust 2 at 105.69564 [yr]
Hit the Planet: dust 20 at 105.91219 [yr]
Hit the Planet: dust 57 at 106.92356 [yr]
Sublimation: dust 39 at 107.50422 [yr]
Sublimation: dust 10 at 107.56470 [yr]
Hit the Planet: dust 76 at 107.59229 [yr]
Hit the Planet: dust 77 at 107.62048 [yr]
Hit the Planet: dust 68 at 107.62706 [yr]
Sublimation: dust 44 at 107.75572 [yr]
Hit the Planet: dust 48 at 107.91854 [yr]
Hit the Planet: dust 70 at 108.02752 [yr]
Sublimation: dust 94 at 108.31596 [yr]
Ejection: dust 25 at 108.54271 [yr]
Ejection: dust 88 at 108.54271 [yr]
Sublimation: dust 32 at 108.86168 [yr]
Sublimation: dust 79 at 109.19330 [yr]
Hit the Planet: dust 5 at 109.27519 [y

read data
--

In [6]:
df_para_rd = pd.read_csv('./test.csv', on_bad_lines='skip')
df_para = df_para_rd.sort_values(by=['m_Star/m_Sun', 'm_Planet/m_J', 'a_p/R_Sun'], ascending = False)
df_para

,m_Star/m_Sun,m_Planet/m_J,a_p/R_Sun,R_sub/R_Sun,R_Planet/R_Sun,beta,a_d_i/a_p,hash,a_initial,e_initial,...,x_p_f_xyz,v_p_f_xyz,x_d_f_xyz,v_d_f_xyz,final_fate,lifetime,a_d_f,e_d_f,vrel_f[km/s],mu
0,1.0,1.0,10.0,5.853188,0.102763,0.15,3.0,2.0,2.087100e+10,0.0,...,[ 6.94809673e+09 -1.77787036e+08 0.00000000e+00],[ 3532.67345715 138060.44232121 0. ],[ 6.88212067e+09 -1.99712632e+08 0.00000000e+00],[ 83412.4284784 144769.54104098 0. ],collision,105.695642,2.347828e+10,0.781940,80.161008,0.973574
1,1.0,1.0,10.0,5.853188,0.102763,0.15,3.0,3.0,2.087100e+10,0.0,...,[6.94972071e+09 9.50704907e+07 0.00000000e+00],[ -1889.07473603 138092.71129326 0. ],[6.93997313e+09 1.63015286e+08 0.00000000e+00],[ 74430.1104253 115122.02321716 0. ],collision,105.272962,8.257483e+09,0.577151,79.701133,0.491709
2,1.0,1.0,10.0,5.853188,0.102763,0.15,3.0,4.0,2.087100e+10,0.0,...,[-3.77740722e+09 -5.83428240e+09 0.00000000e+00],[115928.66933327 -75058.03849223 0. ],[-1.15292947e+09 -3.78611666e+09 0.00000000e+00],[190992.98280374 9492.0820114 0. ],sublimation,111.509988,5.547923e+09,0.432125,0.000000,0.000000
3,1.0,1.0,10.0,5.853188,0.102763,0.15,3.0,5.0,2.087100e+10,0.0,...,[-4.81614294e+09 -5.01122974e+09 0.00000000e+00],[ 99574.40462014 -95697.98043965 0. ],[-4.80410567e+09 -4.94223322e+09 0.00000000e+00],[ 55826.39534483 -163442.20497011 0. ...,collision,109.275186,3.950651e+10,0.864082,80.642224,0.924114
4,1.0,1.0,10.0,5.853188,0.102763,0.15,3.0,6.0,2.087100e+10,0.0,...,[-5.24571721e+09 -4.55961701e+09 0.00000000e+00],[ 90600.7452079 -104233.73006593 0. ...,[-8.04952062e+10 5.35317928e+10 0.00000000e+00],[-46526.66548179 15755.57309378 0. ],ejection,123.618090,-1.706255e+12,1.003891,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,1.0,10.0,5.853188,0.102763,0.15,3.0,97.0,2.087100e+10,0.0,...,[-5.43855033e+09 -4.32779697e+09 0.00000000e+00],[ 85994.42232714 -108065.37301238 0. ...,[-5.40651681e+09 -4.38995461e+09 0.00000000e+00],[140511.76404008 -46673.26812533 0. ],collision,112.858219,1.081833e+10,0.615598,82.104391,0.409428
96,1.0,1.0,10.0,5.853188,0.102763,0.15,3.0,98.0,2.087100e+10,0.0,...,[-8.53565915e+08 6.89775917e+09 0.00000000e+00],[-137060.22233908 -16960.57098225 0. ...,[2.68602390e+09 3.05369416e+09 0.00000000e+00],[-166946.56978847 98809.32442099 0. ...,sublimation,114.908143,6.348099e+09,0.398121,0.000000,0.000000
97,1.0,1.0,10.0,5.853188,0.102763,0.15,3.0,99.0,2.087100e+10,0.0,...,[ 6.87043956e+09 -1.05105501e+09 0.00000000e+00],[ 20884.7293592 136517.37470108 0. ],[ 6.88240920e+09 -9.81278894e+08 0.00000000e+00],[ 91753.46777524 102162.80202506 0. ],collision,104.356755,8.324762e+09,0.572523,78.756681,0.308328
98,1.0,1.0,10.0,5.853188,0.102763,0.15,3.0,100.0,2.087100e+10,0.0,...,[-6.83724652e+09 -1.24888607e+09 0.00000000e+00],[ 24815.68262468 -135857.82057561 0. ...,[-6.24889960e+11 -6.92695177e+10 0.00000000e+00],[-23354.49885262 -4557.77460314 0. ],ejection,104.020101,-5.449367e+11,1.013914,0.000000,0.000000


In [7]:
print(len(df_para[df_para['final_fate']=='collision']))
print(len(df_para[df_para['final_fate']=='sublimation']))
print(len(df_para[df_para['final_fate']=='escape']))
print(len(df_para[df_para['final_fate']=='incomplete']))

48
29
0
1
